In [1]:
# default_exp scraper

# Scraper

> This module is used to scrape https://www.point2homes.com for all land available on Vancouver Island.

In [1]:
#hide
from nbdev.showdoc import *
from lxml.etree import HTML
from selenium import webdriver
from selenium.webdriver.common.by import By
import xmltodict as xd
from time import sleep
from lxml import etree
import pandas as pd
import re

In [2]:
def get_text(xpath):
    return re.sub('<[^<]+?>', '', etree.tostring(xpath, method='html', with_tail=False).decode('ascii')).replace('\n', '').replace('  ', '')

In [3]:
def extract_properties(l):
    price = l.xpath(".//div[@class='price']")[0].text.replace('\n', '')

    name = get_text(l.xpath(".//h3[@class='name']")[0])

    location = l.xpath(".//div[@class='location']")[0].getchildren()[1].text

    size = get_text(l.xpath(".//div[@class='accommodation']")[0])

    description = get_text(l.xpath(".//div[@class='description-text']")[0])

    link = l.xpath(".//a")[0].attrib['href']

    properties = {
        'price': price,
        'name': name,
        'location': location,
        'size': size,
        'description': description,
        'link': link,
    }
    return properties

In [4]:
def read_land_column(lands):
    land_list = []
    for land in lands:
        land_list.append(extract_properties(land))
    return land_list

In [5]:
url = "https://www.thailand-property.com/commercial-property-for-sale?page=1"
browser = webdriver.Firefox()
browser.get(url)

In [6]:
land_data = []
while(1):
    # Read Page

    page_source = HTML(browser.page_source)

    # Extract Land Column
    lands = page_source.xpath("//div[@class='description-block']")

    # Append data
    land_data = land_data + read_land_column(lands)
    
    if not lands:
        print("Likely at the last page. Ending scraping. Goodbye.")
        break
    
    # Report in.
    print(f"Successfully Scraped: {len(lands)}")

    # Go to next page
    url = browser.current_url.split('=')
    url[-1] = str(int(browser.current_url.split('=')[-1])+1)
    url = '='.join(url)
    browser.get(url)
    sleep(10)

Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25


In [7]:
def extract_numbers(s):
    try:
        return float(''.join(re.findall("[-+]?\d*\.\d+|\d+", s)) or 0)
    except:
        return 0
    
def sub_billion(s):
    try:
        return str(int(extract_numbers(''.join(s.split('.'))))) + '0' * (9 - len(str(int(extract_numbers(s.split('.')[1])))))
    except:
        return s.replace('billion','0'*9).strip()

In [8]:
df[df['price'].str.contains('billion')]['price'].apply(sub_billion)

NameError: name 'df' is not defined

In [9]:
df = pd.DataFrame(land_data)
pd.set_option('display.max_colwidth', -1)

In [10]:
df.to_csv("raw_commercial_thailand.csv",index=False)

In [11]:
def explode(s):
    d = {}
    if 'm2' in s:
        t = s.split('m2')
        d['m2'] = t[0].split(' ')[-2]
        if len(t[0].split(' ')) > 3:
            d['beds'] = t[0].split(' ')[0]
            d['baths'] = t[0].split(' ')[1]
    elif len(s.split(' ')) > 1:
        d['beds'] = s.split(' ')[0]
        d['baths'] = s.split(' ')[1]
    return d

In [12]:
from pandas.io.json import json_normalize
df['sizes'] = df['size'].apply(explode)

In [13]:
df = df.merge(json_normalize(df['sizes']), left_index=True, right_index=True).drop(['size','sizes'],axis=1)

In [14]:
df

,price,name,location,description,link,beds,baths,m2
0,"฿ 45,000,000",3 storey building for Sale/Rent! Using space 508 sqm. Soi Inthamara 3 Phahonyothin Rd. BTS Saphankhwai/04-OT-63010.,"Phaya Thai, Bangkok","Good location only 400m from the entrance of Inthamara lane 4 living environment very good ideas for home office or resident 3 storey building on land size 100 sq.wah 20m wide x 20m deep useful space 508 sqm. 4 bedrooms (bathroom and big balcony each room) meeting room, dining room, storage room, kitchen and maid&#8217;s room 5 parking lots Prime Location : BTS Saphankwai, BTS Ari, Big C Saphankwai ,Paolo...",https://www.thailand-property.com/4-bedroom-office-for-sale-or-rent-in-sam-sen-nai-bangkok_4794474,,4,NaN
1,"฿ 7,990,000",Shophouse on Thappraya road for sale and rent.,"Pattaya, Chonburi","4-story Commercial building with air conditioning for rent.Shop, office, 2 bedrooms with sea view. 218 sq.m.,Very good location on busy Thappraya road, Pattaya.Price: 10,750,000 special Discount 26% to 7,990,000 Baht",https://www.thailand-property.com/3-bedroom-shophouse-for-sale-or-rent-in-treetops-pattaya-pratumnak-hill-chonburi_2881665,,3,217
2,"฿ 19,500,000",Business for sale - Beachfront - Bangrak - Koh Samui.,"Ko Samui, Surat Thani","A beach club located in the heart of Bangrak Beach, Koh Samui. This beachfront property has a bar/ restaurant both served by ample kitchen facilities bungalows for rent. The infinity swimming pool is surrounded by wooden decking and provides the perfect location in which to chill away the afternoon until the famous magnificent sunsets on the North East side of the island.&#160;",https://www.thailand-property.com/5-bedroom-commercial-for-sale-in-ko-samui-surat-thani_4155847,,5,NaN
3,"฿ 110,000,000","79 Room Hotel For Sale in Buriram, Thailand.","Mueang Buriram, Buriram","Isan Real Estate Co., Ltd. is proud to offer, for sale, this 79 room renowned Hotel in one of Thailand's major events city, Buriram. Located in the heart of the vibrant Northeastern Thai city, the hotel is a sleek, stylish and modern 'resort in the city' getaway that offers local and International travelers all the comfort, facilities and services demanded for a relaxed, memorable stay. Reviews on...",https://www.thailand-property.com/79-bedroom-hotel-resort-for-sale-in-nai-mueang-buriram_3826649,,79,"6,500"
4,"฿ 15,900,000",4 Bedroom Beautiful Home on Palm Hills Golf Course.,"Hua Hin, Prachuap Khiri Khan","The lovely 4 bedroom home is located within the renowned&#160;Palm Hills Golf Course &amp; Residence&#160;grounds, between Chaam and Hua Hin.This well-fitted home comes with over 1600 square meters of land plot commanding great views over the&#160;generous tropically landscaped gardens towards the foothills of Hua Hin.&#160;The house itself has over 350 square meters of living space over the two floors. Hardwood polished...",https://www.thailand-property.com/4-bedroom-commercial-for-sale-in-hua-hin-prachuap-khiri-khan_4440579,,4,388
...,...,...,...,...,...,...,...,...
1245,"฿ 90,000,000",Luxury Boutique Resort of Condos.,"Hua Hin, Prachuap Khiri Khan","Boutique Resort of 18 x 2 bed condos with 2 storey clubhouse with gym, management office and large pool,...",https://www.thailand-property.com/commercial-for-sale-in-hua-hin-prachuap-khiri-khan_789776,NaN,NaN,10
1246,"฿ 3,990,000","Warehouse / Factory for sale in Thepharak, Samut Prakan","Mueang Samut Prakan, Samut Prakan","Commercial for sale located at Thepharak, Mueang Samut Prakan.",https://www.thailand-property.com/warehouse-factory-for-sale-in-thepharak-samut-prakan_4772431,NaN,NaN,548
1247,"฿ 6,375,561",Artisan Ratchada (&#3629;&#3634;&#3619;&#3660;&#3605;&#3636;&#3595;&#3634;&#3609; &#3619;&#3633;&#3594;&#3604;&#3634;)..,"Huai Khwang, Bangkok",Code : ATS3002Floor : 3Area Size : 91.5 M&#178;PROJECT CONCEPTLocated in the Ratchada district on Thiam Ruammitr...,https://www.thailand-property.com/commercial-for-sale-in-artisan-ratchada-huai-khwang-bangkok_4688492,NaN,NaN,91.5
1

In [15]:
df['price_float (USD)'] = df['price'].apply(sub_billion).apply(extract_numbers) * 0.031 # Baht to USD
df['lot_size_float (ACRE)'] = df['m2'].apply(extract_numbers) / 4046.86 # Square meters to acres
df['size'] = df['m2'] + ' m2'

In [16]:
df

,price,name,location,description,link,beds,baths,m2,price_float (USD),lot_size_float (ACRE),size
0,"฿ 45,000,000",3 storey building for Sale/Rent! Using space 508 sqm. Soi Inthamara 3 Phahonyothin Rd. BTS Saphankhwai/04-OT-63010.,"Phaya Thai, Bangkok","Good location only 400m from the entrance of Inthamara lane 4 living environment very good ideas for home office or resident 3 storey building on land size 100 sq.wah 20m wide x 20m deep useful space 508 sqm. 4 bedrooms (bathroom and big balcony each room) meeting room, dining room, storage room, kitchen and maid&#8217;s room 5 parking lots Prime Location : BTS Saphankwai, BTS Ari, Big C Saphankwai ,Paolo...",https://www.thailand-property.com/4-bedroom-office-for-sale-or-rent-in-sam-sen-nai-bangkok_4794474,,4,NaN,1395000.000,0.000000,NaN
1,"฿ 7,990,000",Shophouse on Thappraya road for sale and rent.,"Pattaya, Chonburi","4-story Commercial building with air conditioning for rent.Shop, office, 2 bedrooms with sea view. 218 sq.m.,Very good location on busy Thappraya road, Pattaya.Price: 10,750,000 special Discount 26% to 7,990,000 Baht",https://www.thailand-property.com/3-bedroom-shophouse-for-sale-or-rent-in-treetops-pattaya-pratumnak-hill-chonburi_2881665,,3,217,247690.000,0.053622,217 m2
2,"฿ 19,500,000",Business for sale - Beachfront - Bangrak - Koh Samui.,"Ko Samui, Surat Thani","A beach club located in the heart of Bangrak Beach, Koh Samui. This beachfront property has a bar/ restaurant both served by ample kitchen facilities bungalows for rent. The infinity swimming pool is surrounded by wooden decking and provides the perfect location in which to chill away the afternoon until the famous magnificent sunsets on the North East side of the island.&#160;",https://www.thailand-property.com/5-bedroom-commercial-for-sale-in-ko-samui-surat-thani_4155847,,5,NaN,604500.000,0.000000,NaN
3,"฿ 110,000,000","79 Room Hotel For Sale in Buriram, Thailand.","Mueang Buriram, Buriram","Isan Real Estate Co., Ltd. is proud to offer, for sale, this 79 room renowned Hotel in one of Thailand's major events city, Buriram. Located in the heart of the vibrant Northeastern Thai city, the hotel is a sleek, stylish and modern 'resort in the city' getaway that offers local and International travelers all the comfort, facilities and services demanded for a relaxed, memorable stay. Reviews on...",https://www.thailand-property.com/79-bedroom-hotel-resort-for-sale-in-nai-mueang-buriram_3826649,,79,"6,500",3410000.000,1.606184,"6,500 m2"
4,"฿ 15,900,000",4 Bedroom Beautiful Home on Palm Hills Golf Course.,"Hua Hin, Prachuap Khiri Khan","The lovely 4 bedroom home is located within the renowned&#160;Palm Hills Golf Course &amp; Residence&#160;grounds, between Chaam and Hua Hin.This well-fitted home comes with over 1600 square meters of land plot commanding great views over the&#160;generous tropically landscaped gardens towards the foothills of Hua Hin.&#160;The house itself has over 350 square meters of living space over the two floors. Hardwood polished...",https://www.thailand-property.com/4-bedroom-commercial-for-sale-in-hua-hin-prachuap-khiri-khan_4440579,,4,388,492900.000,0.095877,388 m2
...,...,...,...,...,...,...,...,...,...,...,...
1245,"฿ 90,000,000",Luxury Boutique Resort of Condos.,"Hua Hin, Prachuap Khiri Khan","Boutique Resort of 18 x 2 bed condos with 2 storey clubhouse with gym, management office and large pool,...",https://www.thailand-property.com/commercial-for-sale-in-hua-hin-prachuap-khiri-khan_789776,NaN,NaN,10,2790000.000,0.002471,10 m2
1246,"฿ 3,990,000","Warehouse / Factory for sale in Thepharak, Samut Prakan","Mueang Samut Prakan, Samut Prakan","Commercial for sale located at Thepharak, Mueang Samut Prakan.",https://www.thailand-property.com/warehouse-factory-for-sale-in-thepharak-samut-prakan_4772431,NaN,NaN,548,123690.000,0.135414,548 m2
1247,"฿ 6,375,561",Artisan Ratchada (&#3629;&#3634;&#3619;&#3660;&#3605;&#3636;&#3595;&#3634;&#3609; &#3619;&#3633;&#3594;&#3604;&#3634;)..,"Huai Khw

In [17]:
df.to_csv("Thailand_Commercial2.csv",index=False)

In [209]:
df.sort_values('price_float (USD)').head(20).replace(0,np.inf)

,price,name,location,description,link,beds,baths,m2,price_float (USD),lot_size_float (ACRE),size
1095,Contact for price,"Shophouse for sale on Manorom, Chai Nat.","Manorom, Chainat",This property of shophouse 3 storey is located on Manorom-Utaithani Road. It is 270 meter from Manorom...,https://www.thailand-property.com/commercial-for-sale-in-khung-samphao-chainat_3872048,79,79,"6,500",inf,1.606184,"6,500 m2"
877,Contact for price,"1 Bedroom Commercial for sale in Lam Pho, Nonthaburi","Bang Bua Thong, Nonthaburi","Commercial for sale of 1,400sqm, consisting of 1 bedroom and 5 bathrooms located at Lam Pho, Bang Bua...",https://www.thailand-property.com/1-bedroom-commercial-for-sale-in-lam-pho-nonthaburi_4281024,79,79,"6,500",inf,1.606184,"6,500 m2"
758,Contact for price,"Land with office building (Standalone) on Kaowang, Petchaburi.","Mueang Phetchaburi, Phetchaburi","The property of land and office building is located on Kaowang ,Phetchaburi it has land size 311 sqw....",https://www.thailand-property.com/commercial-for-sale-in-ban-mo-phetchaburi_3798414,79,79,"6,500",inf,1.606184,"6,500 m2"
631,Contact for price,Shophouse for sale on Wongwien Yai Road.,"Thon Buri, Bangkok","The property of 2 shophouse 4 storey is located on Wongwien Yai Road, Center of Business and Transportation....",https://www.thailand-property.com/commercial-for-sale-in-bang-yi-ruea-bangkok-near-bts-wongwian-yai_3871885,79,79,"6,500",inf,1.606184,"6,500 m2"
622,Contact for price,"4 Bedroom Commercial for sale in Bang Lamphu Lang, Bangkok near BTS Krung Thon Buri","Khlong San, Bangkok","Commercial for sale, consisting of 4 bedrooms and 3 bathrooms located at Bang Lamphu Lang, Khlong San.",https://www.thailand-property.com/4-bedroom-commercial-for-sale-in-bang-lamphu-lang-bangkok-near-bts-krung-thon-buri_4805344,79,79,"6,500",inf,1.606184,"6,500 m2"
1056,Contact for price,Shophouse for sale on Suksawas Road close to Bangpakok 1 Hospital.,"Rat Burana, Bangkok",The property of 4 shophouse 3 storey&#160; is located on Suksawas Road. It is situated close to many...,https://www.thailand-property.com/commercial-for-sale-in-bang-pakok-bangkok_3871249,79,79,"6,500",inf,1.606184,"6,500 m2"
932,Contact for price,"Shophouse for sale on That Choeng Chum, Sakon Nakorn.","Mueang Sakon Nakhon, Sakon Nakhon","The property of 4 storey shophouse is located on Sukkasem Road, That Choeng Chum, Sakon Nakorn which...",https://www.thailand-property.com/commercial-for-sale-in-that-choeng-chum-sakon-nakhon_3871964,79,79,"6,500",inf,1.606184,"6,500 m2"
529,Contact for price,Land with Improvement on Yaowarat-China Town main road.,"Pom Prap Sattru Phai, Bangkok",The property of land and improvement located 450 m. away from MRT Hualumphong on main road Maitri Jit-...,https://www.thailand-property.com/commercial-for-sale-in-pom-prap-bangkok-near-mrt-wat-mangkon_3782964,79,79,"6,500",inf,1.606184,"6,500 m2"
361,Contact for price,"Hua Hin Baan Thai Resort, Thai Lanna style, ready to operate.","Cha am, Phetchaburi","Baan Thai ResortInclude- One big house Lanna style with 5 bedroom include bathroom each, basement can...",https://www.thailand-property.com/hotel-resort-for-sale-in-sam-phraya-phetchaburi_4584807,79,79,"6,500",inf,1.606184,"6,500 m2"
898,Contact for price,"3 Bedroom Commercial for sale in Nong Bua, Nakhon Sawan","Nong Bua, Nakhon Sawan","Commercial for sale of 450sqm, consisting of 3 bedrooms and 6 bathrooms located at Nong Bua, Nakhon Sawan.",https://www.thailand-property.com/3-bedroom-commercial-for-sale-in-nong-bua-nakhon-sawan_4777757,79,79,"6,500",inf,1.606184,"6,500 m2"


In [ ]:
df['']

In [163]:
len(str(int(extract_numbers('1.533 billion'.split('.')[1]))))

3

'1533'

'1533000000'

In [159]:
'0' * (9 - (len('1.5'.split('.')) - 1))

'00000000'

In [190]:
df[df['price'].str.contains('billion')]['price'].apply(sub_billion)

510     1500000000                                               
540     3500000000                                               
570     3500000000                                               
639     3600000000                                               
644     2800000000                                               
675     4990000000                                               
676     ฿ 3 000000000                                            
679     1600000000                                               
716     1800000000                                               
717     ฿ 2 000000000                                            
729     1630000000                                               
732     10710000000                                              
1003    1100000000                                               
1014    ฿ 1 000000000                                            
1016    1907000000                                               
1047    ฿ 

# Working with the data

In [117]:
df = pd.read_csv("Thailand_Commercial.csv")

df['price_float (USD)'] = df['price'].apply(extract_numbers) * 0.031 # Baht to USD
df['lot_size_float (ACRE)'] = df['m2'].apply(extract_numbers) / 4046.86 # Square meters to acres

In [106]:
df

,price,name,location,description,link,beds,baths,m2
0,"฿ 45,000,000",3 storey building for Sale/Rent! Using space 508 sqm. Soi Inthamara 3 Phahonyothin Rd. BTS Saphankhwai/04-OT-63010.,"Phaya Thai, Bangkok","Good location only 400m from the entrance of Inthamara lane 4 living environment very good ideas for home office or resident 3 storey building on land size 100 sq.wah 20m wide x 20m deep useful space 508 sqm. 4 bedrooms (bathroom and big balcony each room) meeting room, dining room, storage room, kitchen and maid&#8217;s room 5 parking lots Prime Location : BTS Saphankwai, BTS Ari, Big C Saphankwai ,Paolo...",https://www.thailand-property.com/4-bedroom-office-for-sale-or-rent-in-sam-sen-nai-bangkok_4794474,NaN,4.0,NaN
1,"฿ 19,500,000",Business for sale - Beachfront - Bangrak - Koh Samui.,"Ko Samui, Surat Thani","A beach club located in the heart of Bangrak Beach, Koh Samui. This beachfront property has a bar/ restaurant both served by ample kitchen facilities bungalows for rent. The infinity swimming pool is surrounded by wooden decking and provides the perfect location in which to chill away the afternoon until the famous magnificent sunsets on the North East side of the island.&#160;",https://www.thailand-property.com/5-bedroom-commercial-for-sale-in-ko-samui-surat-thani_4155847,NaN,5.0,NaN
2,"฿ 7,990,000",Shophouse on Thappraya road for sale and rent.,"Pattaya, Chonburi","4-story Commercial building with air conditioning for rent.Shop, office, 2 bedrooms with sea view. 218 sq.m.,Very good location on busy Thappraya road, Pattaya.Price: 10,750,000 special Discount 26% to 7,990,000 Baht",https://www.thailand-property.com/3-bedroom-shophouse-for-sale-or-rent-in-treetops-pattaya-pratumnak-hill-chonburi_2881665,79.0,79.0,"6,500"
3,"฿ 110,000,000","79 Room Hotel For Sale in Buriram, Thailand.","Mueang Buriram, Buriram","Isan Real Estate Co., Ltd. is proud to offer, for sale, this 79 room renowned Hotel in one of Thailand's major events city, Buriram. Located in the heart of the vibrant Northeastern Thai city, the hotel is a sleek, stylish and modern 'resort in the city' getaway that offers local and International travelers all the comfort, facilities and services demanded for a relaxed, memorable stay. Reviews on...",https://www.thailand-property.com/79-bedroom-hotel-resort-for-sale-in-nai-mueang-buriram_3826649,79.0,79.0,"6,500"
4,"฿ 15,900,000",4 Bedroom Beautiful Home on Palm Hills Golf Course.,"Hua Hin, Prachuap Khiri Khan","The lovely 4 bedroom home is located within the renowned&#160;Palm Hills Golf Course &amp; Residence&#160;grounds, between Chaam and Hua Hin.This well-fitted home comes with over 1600 square meters of land plot commanding great views over the&#160;generous tropically landscaped gardens towards the foothills of Hua Hin.&#160;The house itself has over 350 square meters of living space over the two floors. Hardwood polished...",https://www.thailand-property.com/4-bedroom-commercial-for-sale-in-hua-hin-prachuap-khiri-khan_4440579,79.0,79.0,"6,500"
...,...,...,...,...,...,...,...,...
1245,"฿ 15,000,000","33 Bedroom Commercial for sale in Si Racha, Chonburi","Si Racha, Chonburi","Commercial for sale, consisting of 33 bedrooms located at Si Racha, Chonburi.",https://www.thailand-property.com/33-bedroom-commercial-for-sale-in-si-racha-chonburi_1905172,NaN,33.0,NaN
1246,"฿ 5,500,000",Well Designed Brand New 8 Bed Resort : Reduced in Jan 2017.,"Pran Buri, Prachuap Khiri Khan",Brand New Small Resort located only 1km from the Beach in a Resort destination that features 6 individual...,https://www.thailand-property.com/commercial-for-sale-in-pran-buri-prachuap-khiri-khan_790564,79.0,79.0,"6,500"
1247,"฿ 3,600,000",Shop House near MRT Faichai for sale.,"Bangkok Noi, Bangkok","This shophouse is just 100 m from MRT Faichai station, used as a home office.Nearby:Foodland, Makro,...",https://www.thailand-property.com/commercial-for-sale-in-ban-chang-lo-bangkok-near-mrt-fai-chai_4999580,79.0,79.0,"6,500"
1248,"฿ 4,000,00

In [107]:
t[0].split(' ')

['79', '79', '6,500', '']

In [108]:
d

{'beds': '5', 'baths': '7'}

In [109]:
beds

'79'

In [110]:
baths

'79'

In [111]:
t

['79 79 6,500 ', '']

In [112]:
df['size']

KeyError: 'size'

In [113]:
df

,price,name,location,description,link,beds,baths,m2
0,"฿ 45,000,000",3 storey building for Sale/Rent! Using space 508 sqm. Soi Inthamara 3 Phahonyothin Rd. BTS Saphankhwai/04-OT-63010.,"Phaya Thai, Bangkok","Good location only 400m from the entrance of Inthamara lane 4 living environment very good ideas for home office or resident 3 storey building on land size 100 sq.wah 20m wide x 20m deep useful space 508 sqm. 4 bedrooms (bathroom and big balcony each room) meeting room, dining room, storage room, kitchen and maid&#8217;s room 5 parking lots Prime Location : BTS Saphankwai, BTS Ari, Big C Saphankwai ,Paolo...",https://www.thailand-property.com/4-bedroom-office-for-sale-or-rent-in-sam-sen-nai-bangkok_4794474,NaN,4.0,NaN
1,"฿ 19,500,000",Business for sale - Beachfront - Bangrak - Koh Samui.,"Ko Samui, Surat Thani","A beach club located in the heart of Bangrak Beach, Koh Samui. This beachfront property has a bar/ restaurant both served by ample kitchen facilities bungalows for rent. The infinity swimming pool is surrounded by wooden decking and provides the perfect location in which to chill away the afternoon until the famous magnificent sunsets on the North East side of the island.&#160;",https://www.thailand-property.com/5-bedroom-commercial-for-sale-in-ko-samui-surat-thani_4155847,NaN,5.0,NaN
2,"฿ 7,990,000",Shophouse on Thappraya road for sale and rent.,"Pattaya, Chonburi","4-story Commercial building with air conditioning for rent.Shop, office, 2 bedrooms with sea view. 218 sq.m.,Very good location on busy Thappraya road, Pattaya.Price: 10,750,000 special Discount 26% to 7,990,000 Baht",https://www.thailand-property.com/3-bedroom-shophouse-for-sale-or-rent-in-treetops-pattaya-pratumnak-hill-chonburi_2881665,79.0,79.0,"6,500"
3,"฿ 110,000,000","79 Room Hotel For Sale in Buriram, Thailand.","Mueang Buriram, Buriram","Isan Real Estate Co., Ltd. is proud to offer, for sale, this 79 room renowned Hotel in one of Thailand's major events city, Buriram. Located in the heart of the vibrant Northeastern Thai city, the hotel is a sleek, stylish and modern 'resort in the city' getaway that offers local and International travelers all the comfort, facilities and services demanded for a relaxed, memorable stay. Reviews on...",https://www.thailand-property.com/79-bedroom-hotel-resort-for-sale-in-nai-mueang-buriram_3826649,79.0,79.0,"6,500"
4,"฿ 15,900,000",4 Bedroom Beautiful Home on Palm Hills Golf Course.,"Hua Hin, Prachuap Khiri Khan","The lovely 4 bedroom home is located within the renowned&#160;Palm Hills Golf Course &amp; Residence&#160;grounds, between Chaam and Hua Hin.This well-fitted home comes with over 1600 square meters of land plot commanding great views over the&#160;generous tropically landscaped gardens towards the foothills of Hua Hin.&#160;The house itself has over 350 square meters of living space over the two floors. Hardwood polished...",https://www.thailand-property.com/4-bedroom-commercial-for-sale-in-hua-hin-prachuap-khiri-khan_4440579,79.0,79.0,"6,500"
...,...,...,...,...,...,...,...,...
1245,"฿ 15,000,000","33 Bedroom Commercial for sale in Si Racha, Chonburi","Si Racha, Chonburi","Commercial for sale, consisting of 33 bedrooms located at Si Racha, Chonburi.",https://www.thailand-property.com/33-bedroom-commercial-for-sale-in-si-racha-chonburi_1905172,NaN,33.0,NaN
1246,"฿ 5,500,000",Well Designed Brand New 8 Bed Resort : Reduced in Jan 2017.,"Pran Buri, Prachuap Khiri Khan",Brand New Small Resort located only 1km from the Beach in a Resort destination that features 6 individual...,https://www.thailand-property.com/commercial-for-sale-in-pran-buri-prachuap-khiri-khan_790564,79.0,79.0,"6,500"
1247,"฿ 3,600,000",Shop House near MRT Faichai for sale.,"Bangkok Noi, Bangkok","This shophouse is just 100 m from MRT Faichai station, used as a home office.Nearby:Foodland, Makro,...",https://www.thailand-property.com/commercial-for-sale-in-ban-chang-lo-bangkok-near-mrt-fai-chai_4999580,79.0,79.0,"6,500"
1248,"฿ 4,000,00

In [90]:
df

,price,name,location,description,link,beds,baths,m2,price_float (USD),lot_size_float (ACRE)
0,"฿ 45,000,000",3 storey building for Sale/Rent! Using space 508 sqm. Soi Inthamara 3 Phahonyothin Rd. BTS Saphankhwai/04-OT-63010.,"Phaya Thai, Bangkok","Good location only 400m from the entrance of Inthamara lane 4 living environment very good ideas for home office or resident 3 storey building on land size 100 sq.wah 20m wide x 20m deep useful space 508 sqm. 4 bedrooms (bathroom and big balcony each room) meeting room, dining room, storage room, kitchen and maid&#8217;s room 5 parking lots Prime Location : BTS Saphankwai, BTS Ari, Big C Saphankwai ,Paolo...",https://www.thailand-property.com/4-bedroom-office-for-sale-or-rent-in-sam-sen-nai-bangkok_4794474,NaN,4.0,NaN,1395000.0,0.000000
1,"฿ 19,500,000",Business for sale - Beachfront - Bangrak - Koh Samui.,"Ko Samui, Surat Thani","A beach club located in the heart of Bangrak Beach, Koh Samui. This beachfront property has a bar/ restaurant both served by ample kitchen facilities bungalows for rent. The infinity swimming pool is surrounded by wooden decking and provides the perfect location in which to chill away the afternoon until the famous magnificent sunsets on the North East side of the island.&#160;",https://www.thailand-property.com/5-bedroom-commercial-for-sale-in-ko-samui-surat-thani_4155847,NaN,5.0,NaN,604500.0,0.000000
2,"฿ 7,990,000",Shophouse on Thappraya road for sale and rent.,"Pattaya, Chonburi","4-story Commercial building with air conditioning for rent.Shop, office, 2 bedrooms with sea view. 218 sq.m.,Very good location on busy Thappraya road, Pattaya.Price: 10,750,000 special Discount 26% to 7,990,000 Baht",https://www.thailand-property.com/3-bedroom-shophouse-for-sale-or-rent-in-treetops-pattaya-pratumnak-hill-chonburi_2881665,79.0,79.0,"6,500",247690.0,1.606184
3,"฿ 110,000,000","79 Room Hotel For Sale in Buriram, Thailand.","Mueang Buriram, Buriram","Isan Real Estate Co., Ltd. is proud to offer, for sale, this 79 room renowned Hotel in one of Thailand's major events city, Buriram. Located in the heart of the vibrant Northeastern Thai city, the hotel is a sleek, stylish and modern 'resort in the city' getaway that offers local and International travelers all the comfort, facilities and services demanded for a relaxed, memorable stay. Reviews on...",https://www.thailand-property.com/79-bedroom-hotel-resort-for-sale-in-nai-mueang-buriram_3826649,79.0,79.0,"6,500",3410000.0,1.606184
4,"฿ 15,900,000",4 Bedroom Beautiful Home on Palm Hills Golf Course.,"Hua Hin, Prachuap Khiri Khan","The lovely 4 bedroom home is located within the renowned&#160;Palm Hills Golf Course &amp; Residence&#160;grounds, between Chaam and Hua Hin.This well-fitted home comes with over 1600 square meters of land plot commanding great views over the&#160;generous tropically landscaped gardens towards the foothills of Hua Hin.&#160;The house itself has over 350 square meters of living space over the two floors. Hardwood polished...",https://www.thailand-property.com/4-bedroom-commercial-for-sale-in-hua-hin-prachuap-khiri-khan_4440579,79.0,79.0,"6,500",492900.0,1.606184
...,...,...,...,...,...,...,...,...,...,...
1245,"฿ 15,000,000","33 Bedroom Commercial for sale in Si Racha, Chonburi","Si Racha, Chonburi","Commercial for sale, consisting of 33 bedrooms located at Si Racha, Chonburi.",https://www.thailand-property.com/33-bedroom-commercial-for-sale-in-si-racha-chonburi_1905172,NaN,33.0,NaN,465000.0,0.000000
1246,"฿ 5,500,000",Well Designed Brand New 8 Bed Resort : Reduced in Jan 2017.,"Pran Buri, Prachuap Khiri Khan",Brand New Small Resort located only 1km from the Beach in a Resort destination that features 6 individual...,https://www.thailand-property.com/commercial-for-sale-in-pran-buri-prachuap-khiri-khan_790564,79.0,79.0,"6,500",170500.0,1.606184
1247,"฿ 3,600,000",Shop House near MRT Faichai for sale.,"Bangkok Noi, Bangkok","This shophouse is just 100 m from MRT Faichai station, used as a ho

### Ideas
* For each property give an absolute profit value given a sale at market value
* Time since discount

In [91]:
df['Dollar per Acre'] = df['price_float (USD)'] / df['lot_size_float (ACRE)']
df = df.sort_values('Dollar per Acre')
df['Dollar per Acre'] = df['Dollar per Acre'].apply(lambda x: str(round(x, 2)))

In [92]:
df

,price,name,location,description,link,beds,baths,m2,price_float (USD),lot_size_float (ACRE),Dollar per Acre
877,Contact for price,"1 Bedroom Commercial for sale in Lam Pho, Nonthaburi","Bang Bua Thong, Nonthaburi","Commercial for sale of 1,400sqm, consisting of 1 bedroom and 5 bathrooms located at Lam Pho, Bang Bua...",https://www.thailand-property.com/1-bedroom-commercial-for-sale-in-lam-pho-nonthaburi_4281024,79.0,79.0,"6,500",0.0,1.606184,0.0
1095,Contact for price,"Shophouse for sale on Manorom, Chai Nat.","Manorom, Chainat",This property of shophouse 3 storey is located on Manorom-Utaithani Road. It is 270 meter from Manorom...,https://www.thailand-property.com/commercial-for-sale-in-khung-samphao-chainat_3872048,79.0,79.0,"6,500",0.0,1.606184,0.0
361,Contact for price,"Hua Hin Baan Thai Resort, Thai Lanna style, ready to operate.","Cha am, Phetchaburi","Baan Thai ResortInclude- One big house Lanna style with 5 bedroom include bathroom each, basement can...",https://www.thailand-property.com/hotel-resort-for-sale-in-sam-phraya-phetchaburi_4584807,79.0,79.0,"6,500",0.0,1.606184,0.0
529,Contact for price,Land with Improvement on Yaowarat-China Town main road.,"Pom Prap Sattru Phai, Bangkok",The property of land and improvement located 450 m. away from MRT Hualumphong on main road Maitri Jit-...,https://www.thailand-property.com/commercial-for-sale-in-pom-prap-bangkok-near-mrt-wat-mangkon_3782964,79.0,79.0,"6,500",0.0,1.606184,0.0
437,Contact for price,Land with buildings 50 rai Romklao Rd near Suvarnabhumi.,"Lat Krabang, Bangkok",Land with buildings 50-1-32 Rai for sale on Romklao road frontageProperty DetailLand area:50-1-31.8 rai...,https://www.thailand-property.com/commercial-for-sale-in-khlong-sam-prawet-bangkok_4693975,79.0,79.0,"6,500",0.0,1.606184,0.0
...,...,...,...,...,...,...,...,...,...,...,...
789,"฿ 15,000,000",Guesthouse for sale at Na Jomtien.,"Pattaya, Chonburi","6 Buildings, 11 bedrooms (including 2 dormitories, 13 bathrooms with hot water), all rooms, except dormitories,...",https://www.thailand-property.com/commercial-for-sale-in-pattaya-chonburi_3995788,NaN,13.0,NaN,465000.0,0.000000,inf
788,"฿ 70,000,000",33 Rooms Resort in Phuket with No Key Money.,"Mueang Phuket, Phuket","A 33 Rooms Resort with an area on about 2 rai, is available for Lease in Phuket. The Resort is short...",https://www.thailand-property.com/33-bedroom-hotel-resort-for-sale-or-rent-in-nai-harn-phuket_4041825,NaN,33.0,NaN,2170000.0,0.000000,inf
787,"฿ 3,000,000",Business For Sale.,"Pattaya, Chonburi",Profitable Massage Business in fantastic prominent location for sale in the heart of Central PattayaMany...,https://www.thailand-property.com/4-bedroom-commercial-for-sale-in-pattaya-chonburi_4224998,NaN,4.0,NaN,93000.0,0.000000,inf
815,"฿ 260,110,000","Commercial for sale in Krathum Lom, Nakhon Pathom","Sam Phran, Nakhon Pathom",The factory is only 100 meters away from Phetkasem . There are 4 factories available for sale at a special...,https://www.thailand-property.com/commercial-for-sale-in-krathum-lom-nakhon-pathom_5040467,NaN,NaN,NaN,8063410.0,0.000000,inf


In [226]:
df.head(100).to_csv('ThailandTop100.csv', index=False)

In [250]:
df['Dollar per Acre'] = df['Dollar per Acre'].apply(float)

In [251]:
df_regions = df.head(100)[df.head(100)['location'].str.contains(', ')].copy(deep=True)

In [252]:
df_regions['region'] = df_regions['location'].apply(lambda x: x.split(',')[1])

In [253]:
df_regions

,price,name,location,size,description,link,price_float (USD),lot_size_float (ACRE),Dollar per Acre,region
969,"฿ 4,000,000",Exotic Land Plot with Feature Pond.,"Hua Hin, Prachuap Khiri Khan","1,920,000 m2/ 1,200 Rai",Beautiful piece of Land that can have its own ...,https://www.thailand-property.com/land-for-sal...,124000.0,474.441913,261.36,Prachuap Khiri Khan
6,"฿ 350,000","Land for sale in Pak Chong, Ratchaburi","Chom Bueng, Ratchaburi","133,236 m2/ 83 Rai 1 Ngan 9 Sq.W.","Land for sale located at Pak Chong, Chom Bueng.",https://www.thailand-property.com/land-for-sal...,10850.0,32.923303,329.55,Ratchaburi
1135,"฿ 5,000,000",Land for with Resort Building for Sale or Ren...,"Mae On, Chiang Mai","6 7 2,952 m2/ 1 Rai 3 Ngan 38 Sq.W.",A plot of Land of 738 square wah (2952 square ...,https://www.thailand-property.com/6-bedroom-la...,155000.0,166.289914,932.11,Chiang Mai
33,"฿ 600,000",Land for sale 50 rai in HuaHin.,"Hua Hin, Prachuap Khiri Khan","80,000 m2/ 50 Rai","Land for sale 50 rai, 6.0 K / 1 rai Land tota...",https://www.thailand-property.com/land-for-sal...,18600.0,19.768413,940.89,Prachuap Khiri Khan
397,"฿ 2,100,000","2 Bedroom Land for sale in Phak Kha, Sa Kaeo","Watthana Nakhon, Sa Kaeo","2 1 1,296 m2","Land for sale of 1,296sqm, consisting of 2 bed...",https://www.thailand-property.com/2-bedroom-la...,65100.0,52.212333,1246.83,Sa Kaeo
...,...,...,...,...,...,...,...,...,...,...
703,"฿ 3,096,000","Land for sale in Pa Sakae, Suphan Buri","Doem Bang Nang Buat, Suphan Buri","17,258.4 m2/ 10 Rai 3 Ngan 14.6 Sq.W.","Land for sale located at Pa Sakae, Doem Bang N...",https://www.thailand-property.com/land-for-sal...,95976.0,4.264640,22505.07,Suphan Buri
187,"฿ 1,400,000","Land for sale in Ban Fang, Khon Kaen","Ban Fang, Khon Kaen","7,768 m2/ 4 Rai 3 Ngan 42 Sq.W.","Land for sale located at Ban Fang, Khon Kaen.",https://www.thailand-property.com/land-for-sal...,43400.0,1.919513,22609.90,Khon Kaen
485,"฿ 2,500,000",Cheap Land for sale Adjacent to T-Park Indust...,"Si Racha, Chonburi","13,828 m2/ 8 Rai 2 Ngan 57 Sq.W.","Land for sale in purple, striped, adjacent to ...",https://www.thailand-property.com/land-for-sal...,77500.0,3.416970,22680.91,Chonburi
339,"฿ 1,900,000","Land for sale in Ban Khuan, Chumphon","Lang Suan, Chumphon","10,368 m2","Land for sale of 10,368sqm located at Ban Khua...",https://www.thailand-property.com/land-for-sal...,58900.0,2.561986,22989.97,Chumphon


In [263]:
a = df_regions.groupby('region').sum()[['price_float (USD)', 'lot_size_float (ACRE)']]

In [264]:
b = df_regions.groupby('region').count()[['location']]

In [265]:
c = df_regions.groupby('region').mean()[['Dollar per Acre']]

In [268]:
info = a.merge(b, left_index=True, right_index=True).merge(c, left_index=True, right_index=True)[['location','price_float (USD)', 'lot_size_float (ACRE)', 'Dollar per Acre']]

In [272]:
info.columns = ['Discounted Properties', 'Total Price (USD)', 'Total Size (ACRE)', 'Mean Dollar Per Acre on Discounted Properties']

In [273]:
info

,Discounted Properties,Total Price (USD),Total Size (ACRE),Mean Dollar Per Acre on Discounted Properties
region,,,,
Amnat Charoen,1,49600.000,3.162946,15681.580000
Chanthaburi,1,93000.000,4.513129,20606.550000
Chiang Mai,3,383625.000,180.918539,10802.040000
Chiang Rai,2,116808.000,14.898464,11034.570000
Chonburi,13,1090270.000,179.298518,10388.151538
Chumphon,1,58900.000,2.561986,22989.970000
Kamphaeng Phet,1,46500.000,3.953683,11761.190000
Khon Kaen,8,574324.600,97.347573,12109.157500
Lamphun,1,155000.000,8.556560,18114.760000


In [283]:
df.head()

,price,name,location,size,description,link,price_float (USD),lot_size_float (ACRE),Dollar per Acre
969,"฿ 4,000,000",Exotic Land Plot with Feature Pond.,"Hua Hin, Prachuap Khiri Khan","1,920,000 m2/ 1,200 Rai",Beautiful piece of Land that can have its own ...,https://www.thailand-property.com/land-for-sal...,124000.0,474.441913,261.36
6,"฿ 350,000","Land for sale in Pak Chong, Ratchaburi","Chom Bueng, Ratchaburi","133,236 m2/ 83 Rai 1 Ngan 9 Sq.W.","Land for sale located at Pak Chong, Chom Bueng.",https://www.thailand-property.com/land-for-sal...,10850.0,32.923303,329.55
1135,"฿ 5,000,000",Land for with Resort Building for Sale or Ren...,"Mae On, Chiang Mai","6 7 2,952 m2/ 1 Rai 3 Ngan 38 Sq.W.",A plot of Land of 738 square wah (2952 square ...,https://www.thailand-property.com/6-bedroom-la...,155000.0,166.289914,932.11
33,"฿ 600,000",Land for sale 50 rai in HuaHin.,"Hua Hin, Prachuap Khiri Khan","80,000 m2/ 50 Rai","Land for sale 50 rai, 6.0 K / 1 rai Land tota...",https://www.thailand-property.com/land-for-sal...,18600.0,19.768413,940.89
397,"฿ 2,100,000","2 Bedroom Land for sale in Phak Kha, Sa Kaeo","Watthana Nakhon, Sa Kaeo","2 1 1,296 m2","Land for sale of 1,296sqm, consisting of 2 bed...",https://www.thailand-property.com/2-bedroom-la...,65100.0,52.212333,1246.83


In [285]:
df['Dollar per Acre'].mean() * 0.01

8680.923866278028

In [286]:
df_discounted = df[df['Dollar per Acre'] < df['Dollar per Acre'].mean() * 0.01]

In [288]:
len(df_discounted)

40

In [289]:
len(df)

1115

In [284]:
df['Dollar per Acre'].mean()

868092.3866278027

### We can filter by discount per region, that might be more interesting.
We can also make a panel widget that allows us to slide our discount threshold.
Additionally, we can render a map showing the regions. We can encode dollar per acre per property on the map.